## Exercise 1: An introductory lesson on using Google Earth Engine

*The exercise below follows [**Chapter 15, Exercise 1**](https://learning.nceas.ucsb.edu/2022-09-arctic/sections/15-google-earth-engine.html#exercise-1-an-introductory-lesson-on-using-google-earth-engine) from the book, [Scalable and Computationally Reproducible Approaches to Arctic Research](https://learning.nceas.ucsb.edu/2022-09-arctic/). Code cells are intentially left blank so that participants may code along. Feel free to reference [the book](https://learning.nceas.ucsb.edu/2022-09-arctic/sections/15-google-earth-engine.html#exercise-1-an-introductory-lesson-on-using-google-earth-engine) to check your work against the example code provided.*

### **Part i.** Setup

1. Create a Google Earth Engine account (if you haven't already done so) -- refer back to the [Preface](https://learning.nceas.ucsb.edu/2022-09-arctic/#create-a-free-google-earth-engine-gee-account) for instructions on how to do so

2. Load libraries

In [1]:
# import `ee` and `geemap` libraries
import ee #earth engine api
import geemap #specifically for making maps
import pandas as pd

3. Authenticate your GEE account

- In order to begin using GEE, you’ll need to connect your environment (`scomp`) to the authentication credentials associated with your Google account. This will need to be done each time you connect to GEE, (but only be done once per session).

In [2]:
# authenticate
ee.Authenticate() # triggers the authentication process


Successfully saved authorization token.


This should launch a browser window where you can login with your Google account to the Google Earth Engine Authenticator. Following the prompts will generate a code, which you’ll then need to copy and paste into the VS Code command palette (at the top of the IDE). This will be saved as an authentication token so you won’t need to go through this process again until the next time you start a new session. The browser-based authentication steps will look something like this:

- **Notebook Authenticator:** choose an active Google account and Cloud Project (you may have to create one if this is your first time authenticating) and click “Generate Token”  
- **Choose an account:** if prompted, select the same Google account as above  
- **Google hasn’t verified this app:** You may be temped to click the blue “Back to saftey” button, but don’t! Click “Continue”  
- **Select what Earth Engine Notebook Client can access:** click both check boxes, then “Continue”  
- **Copy your authorization code** to your clipboard to paste into the VS Code command palette

4. Lastly, initialize. This verifies that valid credentials have been created and populates the Python client library with methods that the backend server supports.

In [3]:
# intialize
ee.Initialize() 

If successful, you're now ready to begin working with Google Earth Engine!

### **Part ii.** Explore the ERA5 Daily Aggregates Data

We’ll be using the ERA5 daily aggregates reanalysis dataset, produced by the [European Centre for Medium-Range Weather Forecasts](https://www.ecmwf.int/) (ECMWF), found [here](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY), which models atmospheric weather observations.

Take a few moments to explore the metadata record for this dataset. You’ll notice that it includes a bunch of important information, including:

- **Dataset Availability:** the date range
- **Dataset Provider:** where the data come from
- **Earth Engine Snippet:** a code snippet used for loading the dataset
- **Description (tab):** get to know a bit about the data
- **Bands (tab):** the variables present in the dataset; each band has its own name, data type, scale, mask and projection
- **Image Properties:** metadata available for each image band
- **Example Code:** a script to load and visualize ERA5 climate reanalysis parameters in Google Earth Engine (JavaScript)

### **Part iii.** Visualize global precipitation using ERA5 Daily Aggregate data

*Content for this section was adapted from Dr. Sam Stevenson’s [Visualizing global precipitation using Google Earth Engine](https://github.com/samanthastevenson/EDS220_Fall2021/blob/main/Week1_ERA5Maps_GEE.ipynb) lesson, given in her [EDS 220 course](https://samanthastevenson.github.io/EDS220_site/) in Fall 2021.*

1. Create an interactive basemap

- The default basemap is (you guessed it) Google Maps. Let's create an empty Google Map that you can manipulate just like you would in the typical Google Maps interface. Do this using the `Map` method from the `geemap` library. We’ll also center the map at a specified latitude and longitude (here, 40N, 100E), set a zoom level, and save our map as an object called `myMap`.

In [4]:
# create a basemap called `myMap`
myMap = geemap.Map(center = [60, -151], zoom = 4)
myMap

Map(center=[60, -151], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

2. Load the ERA5 Image Collection from GEE

- Next, we need to tell GEE what data we want to layer on top of our basemap. The `ImageCollection` method extracts a set of individual images that satisfies some criterion that you pass to GEE through the `ee` package. This is stored as an **ImageCollection** object which can be filtered and processed in various ways. We can pass the `ImageCollction` method agruments to tell GEE which data we want to retrieve. Below, we retrieve all daily ERA5 data.

In [5]:
# load ERA5 data and save as object called `weatherData`
weatherData = ee.ImageCollection('ECMWF/ERA5/DAILY')

3. Select an image to plot

- To plot a map over our Google Maps basemap, we need an *Image* rather than an *ImageCollection*. ERA5 contains many different climate variables – explore which variables the dataset contains under the [Bands](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY#bands) tab. We’ll use the `select` method to choose the parameter(s) we’re interested in from our `weatherData` object. Let’s select the `total_precipitation` band.

In [6]:
# select desired bands (total_preciptation); save as object called `precip`
precip = weatherData.select('total_precipitation')

- We can look at our `precip` object metadata using the `print` method to see that it’s still an *ImageCollection*.

In [7]:
# print `precip`
print(precip)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Image.select",
              "arguments": {
                "bandSelectors": {
                  "constantValue": [
                    "total_precipitation"
                  ]
                },
                "input": {
                  "argumentReference": "_MAPPING_VAR_0_0"
                }
              }
            }
          }
        }
      },
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.load",
          "arguments": {
            "id": {
              "constantValue": "ECMWF/ERA5/DAILY"
            }
          }
        }
      }
    }
  }
})


- Let’s say that we want to look at data for a particular time of interest – e.g. January 1, 2019 - December 31, 2019. We can apply the `filterDate` method to our selected `total_precipitation` parameter to filter for data from our chosen date range. We can also apply the `mean` method, which takes whatever precedes it and calculates the average. 

In [8]:
# initial date of interest (inclusive)
i_date = '2019-01-01'

# final data of interest (exclusive)
f_date = '2020-01-01'

# select apporpriate bands (total_preciptation), dates, and calculate mean precipitation across that date range
precip = weatherData.select('total_precipitation').filterDate(i_date,f_date).sum()

- Use the print method again to check out your new `precip` object – notice that it’s now an `ee.Image` (rather than `ee.ImageCollection`) and the start and end date values are as we specified.

In [9]:
# print `precip` again
print(precip)

ee.Image({
  "functionInvocationValue": {
    "functionName": "reduce.sum",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_0_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Image.select",
                          "arguments": {
                            "bandSelectors": {
                              "constantValue": [
                                "total_precipitation"
                              ]
                            },
                            "i

4. Add the precipitation Image to the basemap

- We can first use the `setCenter` method to tell the map where to center itself (here, we'll center it over Cook Inlet, Alaska). It takes the longitude and latitude as the first two arguments, followed by the zoom level.

In [10]:
# center map over Cook Inlet, AK (lat = 60, lon = -151)
myMap = geemap.Map(center = [60, -151], zoom = 4)
myMap

Map(center=[60, -151], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

- Next, set a color palette to use when plotting the data layer. The following is a palette specified for ERA5 precipitation data (scroll down to the example code, available on the [landing page for the ERA5 metadata](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY) in the Earth Engine Data Catelog). We can adjust the max value to change the range of pixel values to which the palette should be applied – this will make our colors stand out a bit more when we layer our precipitation data on our basemap, below.

In [11]:
# create a color palette for our precipitation map; save it as an object called `precip_palette`
precip_palette = {
    'min':0,
    'max':5,
    'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

- Finally, plot our filtered data, `precip`, on top of our basemap using the `addLayer` method. We’ll also pass it our visualization parameters (colors and ranges stored in `precip_palette`, the name of the data field, `total precipitation`, and opacity (so that we can see the basemap underneath).

In [34]:
# add precipitation data to our basemap centered over Cook Inlet, AK
myMap.addLayer(precip,precip_palette, "total precipitation", opacity = .7)
myMap

Map(bottom=2514.0, center=[63.4308891222331, -131.90205989926682], controls=(WidgetControl(options=['position'…

In [13]:
# initialize data frame of points
data = [['anc', 61.2, -150.1 ], ['fai', 64.8, -147.6], ['jun', 58.3, -134.4]]
  
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['loc', 'lat', 'lon'])

In [14]:
# convert data frame to list of GEE features
features=[]

#iterating through the rows and creating a feature list from the point coordinates
for index, row in df.iterrows():
    p_geom = ee.Geometry.Point([row['lon'], row['lat']]) #pulling the lat and lon for each row
    # construct attributes for each row
    p_props = dict(row) #dictionary of each row
    p_feature = ee.Feature(p_geom, p_props) #initializing an eefeature
    features.append(p_feature) #adding each feature to the list

In [15]:
#changing the features list to something ee can use, a feature collection
ee_fc = ee.FeatureCollection(features)

In [16]:
#sampling the values for these points, scale is saying at a 30 meter resolution?, the most zoomed in
res = precip.sampleRegions(collection = ee_fc, scale = 30)

In [17]:
#making a df of the results
res_df = geemap.ee_to_pandas(res)
res_df

,loc,lon,lat,total_precipitation
0,anc,-150.1,61.2,0.790397
1,fai,-147.6,64.8,0.566299
2,jun,-134.4,58.3,2.779951


In [18]:
#loading burn data from ee data
burnData = ee.ImageCollection("MODIS/061/MCD64A1")

In [29]:
myMap3 = geemap.Map(center = [66,-145], zoom = 7)

In [35]:
burn = burnData.select('BurnDate')

burn_palette = {
  "min": 30.0,
  "max": 341.0,
  "palette": ['4e0400', '951003', 'c61503', 'ff1901']}

myMap.addLayer(burn, burn_palette, "Burn", opacity = .7)
myMap

Map(bottom=2514.0, center=[63.4308891222331, -131.90205989926682], controls=(WidgetControl(options=['position'…

In [21]:
burn = ee.ImageCollection("ESA/CCI/FireCCI/5_1")
burn_2020 = burn.select('BurnDate').mean()
fire_pal = {
  'min': 1,
  'max': 366,
  'palette': [
    'ff0000', 'fd4100', 'fb8200', 'f9c400', 'f2ff00', 'b6ff05',
    '7aff0a', '3eff0f', '02ff15', '00ff55', '00ff99', '00ffdd',
    '00ddff', '0098ff', '0052ff', '0210ff', '3a0dfb', '7209f6',
    'a905f1', 'e102ed', 'ff00cc', 'ff0089', 'ff0047', 'ff0004'
  ]
}


myMap2 = geemap.Map(center = [66,-145], zoom = 7)

myMap2.addLayer(burn_2020, fire_pal, "Burned area with date burned", opacity = 1)
legend_dict = {
    "January 1" : '#ff0000',
    "March 1" : '#7aff0a',
    "June 1" : '#00ddff',
    "Sept 1" : '#a905f1'
}

myMap2.add_legend(legend_title="Burn Day", legend_dict=legend_dict)
myMap2


Map(center=[66, -145], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

TypeError: add_legend() got multiple values for argument 'legend_dict'